In [21]:
import sys
# Version 3.8 de Python requerida
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install contexto
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [40]:
from contexto.limpieza import limpieza_texto, lista_stopwords, remover_stopwords

import pandas as pd

In [22]:
df = pd.read_excel('cat_6716.xlsx')

In [24]:
df.head()

,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequí...",6
1,Hay una gran cantidad de literatura sobre Aust...,6
2,"Los procesos de descentralización, emprendidos...",6
3,Esto puede tener consecuencias sustanciales pa...,6
4,La función de beneficio también incorpora pará...,6


In [35]:
# Pasar todo el texto a minúsculas
# Quitar signos de puntuación
# Quitar stopwords (palabras y/o expresiones). Para esto, se pueden pasar directamente las listas de palabras y expresiones a quitar, o se puede pasar un archivo que contenga esta información.
# Quitar palabras de una longitud menor a n caracteres (configurable)
# Quitar números (configurable)
# Quitar acentos (configurable)

df['Textos_espanol'] = df['Textos_espanol'].apply(limpieza_texto)
df.head()

,Textos_espanol,sdg
0,es importante destacar que en un ano de sequia...,6
1,hay una gran cantidad de literatura sobre aust...,6
2,los procesos de descentralizacion emprendidos ...,6
3,esto puede tener consecuencias sustanciales pa...,6
4,la funcion de beneficio tambien incorpora para...,6


In [38]:
print(df['Textos_espanol'][0])

es importante destacar que en un ano de sequia se espera que disminuyan todos los aportes pero que aumente la demanda de agua de los cultivos y de la vegetacion lo que provocara una reduccion del caudal de los rios que estan conectados hidrologicamente con el acuifero asi como una menor disponibilidad de agua para otros ecosistemas dependientes de las aguas subterraneas estas externalidades espaciales pueden incluir el agotamiento de los cursos de agua y el descenso del nivel freatico local por ejemplo el bombeo continuo del pozo b provocara el agotamiento de los cursos de agua y la reduccion de su caudal


In [61]:
stopwords = lista_stopwords()

def remove_stopwords_from_text(text):
    return remover_stopwords(text, stopwords)

df['Textos_espanol'] = df['Textos_espanol'].apply(lambda x: remove_stopwords_from_text(x))

df.head()

,Textos_espanol,sdg
0,importante destacar ano sequia espera disminuy...,6
1,gran cantidad literatura australia area sugier...,6
2,procesos descentralizacion emprendidos serie a...,6
3,puede tener consecuencias sustanciales calidad...,6
4,funcion beneficio tambien incorpora parametros...,6


In [63]:
print(df['Textos_espanol'][0])

importante destacar ano sequia espera disminuyan aportes aumente demanda agua cultivos vegetacion provocara reduccion caudal rios estan conectados hidrologicamente acuifero menor disponibilidad agua ecosistemas dependientes aguas subterraneas externalidades espaciales pueden incluir agotamiento cursos agua descenso nivel freatico local ejemplo bombeo continuo pozo provocara agotamiento cursos agua reduccion caudal
